# Battleships

## Planning

- standard version & extended version (if we have time)
    - standard
        - normal rules
        - 2 human players
        - basic display (jupyter output)
    - extended
        - graphics display (turtle?)
        - hard mode ("Salvo")
        - computer player
            - random guess
            - if last guess == hit, random guess within closer range...
        - different size board
        - different size ships(?)


Standard:

- board
    - 10x10
    - 4 boards (player_1_sea, player_1_radar, player_2_sea, player_2_radar)

- ships
    - number/size has to make sense for the board dimensions
    - standard:
        - Carrier (5)
        - Battelship (4)
        - Cruiser (3)
        - Submarine (3)
        - Destroyer (2)

- turn
    - guess one coordinate on board
        - outputs result (miss / hit + ship if hit)
            - sunk if ship gone
        - displays new board (updated pegs)

- display
    - can only see your board

Functions:

- display_board
    - input lists
    - output (print) sea and radar

- player_names
    - input string

- place_ships
    - choose ship
    - choose starting coordinate
    - choose direction (udlr)
    - calls check_space_free (iterates for all coords in selection)
    - updates ship variables

- update board

- check_space_free

- choose_first_player
    - random (0, 1)

- fire_shot
    - choose location to fire at
    - update board/list

- check_hit
    - returns result of shot
    - update ship count

- check_ship_status
    - checks if 

- save_shot
    - remove available firing locations
    - decide whether can play in same square again

- ready_for_turn
    - when turn ended, hide board
    - ask if ready
    - when ready, show other player board

- winning_condition
    - ship count == 0

- replay

Game Logic

- initiate empty board
- ask for player names
- decide who goes first

- player 1 places ships
    - display_board per ship placed
    - ready_for_turn
- player 2 places ships
    - display_board per ship placed
    - ready_for_turn

- while not winning_condition:
    - player 1 fire_shot
        - display board
        - (update board)
        - check_hit
        - check_ship_status
        - check winning_condition
        - display board
        - ready_for_turn
    - player 2 fire_shot (same logic as player 1)

- display result
    - display both boards (?)

- ask for replay

## Functions

In [13]:
# Library imports
import random
from IPython.display import clear_output
import ipywidgets as widgets


In [61]:
# Initial variables

letters = ["A","B","C","D","E","F","G","H","I","J"]

# Radar squares
radar_empty = "   "
radar_boats = [" D ", " C ", " S ", " B ", " A "]
radar_hit = " x "
radar_miss = " o "

# Sea squares
sea_empty = "~~~"
sea_boats = ["~D~", "~C~", "~S~", "~B~", "~A~"]
sea_hit = "~x~"
sea_miss = "~o~"

# Boat types
boat_types = {"Aircraft carrier (A)" : (5, "~A~"), 
              "Battleship (B)" : (4, "~B~"), 
              "Submarine (S)" : (3, "~S~"), 
              "Cruiser (C)" : (3, "~C~"), 
              "Destroyer (D)" : (2, "~D~")}


player_1_sunk_boats = []
player_2_sunk_boats = []


# Lists of player boat locations
player_1_boat_coords = []
player_2_boat_coords = []

# To be included in game logic?
sea_lst_p1 = [sea_empty] * 101  
radar_lst_p1 = [radar_empty] * 101
sea_lst_p2 = [sea_empty] * 101  
radar_lst_p2 = [radar_empty] * 101

# Lists of past guesses
player_1_guesses = []
player_2_guesses = []

In [15]:
player_1_remaining_boats

['Aircraft carrier (A)',
 'Battleship (B)',
 'Submarine (S)',
 'Cruiser (C)',
 'Destroyer (D)']

In [35]:
def player_names():
    """Asks for the names of Players 1 and 2 and returns both in a tuple."""
    player_1 = input("type your name:")
    player_2 = input("type your name:")
    return player_1, player_2

def choose_first_player():
    """Randomly selects which player has the first turn. Returns 'p1' or 'p2'."""
    if random.randint(0,1) == 0:
        return "p1"
    else:
        return "p2"

def display_boards(radar_lst, sea_lst):
    """Prints the radar_lst and sea_lst lists in the format of Battleships boards, with the letters list
    denoting the rows."""
    
    clear_output()
    
    print("Radar zone\n")
    print("     1   2   3   4   5   6   7   8   9   10")
    for index, i in enumerate(radar_lst):
        if index == 0:
            pass
        elif index % 10 == 1:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 0:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")
            
    print("\nSea zone\n")
    print("     1   2   3   4   5   6   7   8   9   10")
    for index, i in enumerate(sea_lst):
        if index == 0:
            pass
        elif index % 10 == 1:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 0:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")

In [17]:
practice = sea_lst_p1
practice[52] = sea_boats[3]
practice[62] = sea_boats[3]
practice[72] = sea_boats[3]
practice[82] = sea_boats[3]

display_boards(radar_lst_p1, practice)

Radar zone

     1   2   3   4   5   6   7   8   9   10
A: |   |   |   |   |   |   |   |   |   |   |
B: |   |   |   |   |   |   |   |   |   |   |
C: |   |   |   |   |   |   |   |   |   |   |
D: |   |   |   |   |   |   |   |   |   |   |
E: |   |   |   |   |   |   |   |   |   |   |
F: |   |   |   |   |   |   |   |   |   |   |
G: |   |   |   |   |   |   |   |   |   |   |
H: |   |   |   |   |   |   |   |   |   |   |
I: |   |   |   |   |   |   |   |   |   |   |
J: |   |   |   |   |   |   |   |   |   |   |

Sea zone

     1   2   3   4   5   6   7   8   9   10
A: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
B: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
C: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
D: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
E: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
F: |~~~|~B~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
G: |~~~|~B~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
H: |~~~|~B~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
I: |~~~|~B~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
J: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~

In [18]:
def update_board_placement(sea_lst, boat_coords, symbol):
    """Updates the board after the placement of a boat, taking a list of boat_coords as input."""
    for coord in boat_coords:
        sea_lst[coord] = symbol

In [19]:
def update_board_combat(radar_lst, sea_lst, position, flag):
    """Taking the position and flag results from a turn, this function updates the radar_lst and 
    sea_lst lists that make up the board."""
    radar_lst[position] = flag
    sea_lst[position] = flag

In [20]:
def enforce_player_coord(coord):
    """Ensures a player coordinate input is in the valid 'A1' - 'J10' format. Returns coord once in the correct format."""
    # Ensure second item in coord is a number
    try:
        int(coord[1])
    except (ValueError, IndexError):
        coord = input("Enter coordinates (in the format 'A1'): ").upper()

    # Ensure coord is 2-3 characters long (and ends in '10') if 3 characters, starts with a letter in letters and
    # ends in a number 1-10
    while len(coord) > 3 or len(coord) < 2 or \
    (len(coord) == 3 and coord[1:3] != "10") or \
    coord[0] not in letters or int(coord[1]) not in range(1, 9):
        coord = input("Enter coordinates (in the format 'A1'): ").upper()
     
    return coord.upper()

In [73]:
def fire_shot(player_guesses):
    """Calls enforce_player_coord() to prompt player to enter a guess and enforce validity.
    If coordinates have been guessed by the player already, asks for another guess.
    If the guess is valid, adds the guess to player_guesses and returns the 
    list index ('position') of the player guess."""
    
    guess = ""
    guess = enforce_player_coord(guess)
    
    # Checks if guess has already been guessed in a previous turn
    while guess in player_guesses:
        print("Coordinates already guessed!")
        guess = input("Enter coordinates (in the format 'A1'): ").upper() 
    
    player_guesses.append(guess) # adds guess to guess history
    position = letters.index(guess[0]) * 10 + int(guess[1:])
    return position

In [22]:
test_guesses = ["A5", "G6"]

position = fire_shot(test_guesses, letters)

print(test_guesses)
print(position)

Enter coordinates (in the format 'A1'):  D4


['A5', 'G6', 'D4']
34


In [41]:
def check_space_free(sea_lst, start_coord, direction, length):
    """Checks if all required spaces for boat placement are free and in range.
    Returns coordinates if valid, otherwise returns False."""
    
    # use length of boat and direction to find all coords it will cover
    # if any of those coords are not sea empty, return False
        # also handle out of range issues
    # else return coords

    if sea_lst[start_coord] != sea_empty:
        return False
    
    boat_coords = [start_coord]
    row = start_coord//10
    if start_coord%10 == 0:
        row -= 1
                
    if direction in ["L", "R"]:
        d = 1
        if direction == "L":
            d = -1
        for i in range(1, length):
            next_coord = start_coord + (i * d)
            
            if (next_coord - 1) // 10 != row:
                return False
            elif sea_lst[next_coord] != sea_empty:
#                 return "Sea already occupied here"
                return False
            else:
                boat_coords.append(next_coord)
        return boat_coords
    
    elif direction in ["U", "D"]:
        d = 1
        if direction == "U":
            d = -1
        for i in range(1, length):
            next_coord = start_coord + (i * 10 * d)
            if next_coord not in range(1, 101):
#                 return "Ship outside board"
                return False
            elif sea_lst[next_coord] != sea_empty:
#                 return "Sea already occupied here"
                return False
            else:
                boat_coords.append(next_coord)
        return boat_coords
    
    else:
        return False

In [24]:
def coord_string_to_digit(coord):
    return letters.index(coord[0]) * 10 + int(coord[1:])

In [60]:
def place_boats(radar_lst, sea_lst, player_boat_coords):
    """Prompts user to place boats in descending order of length, by entering starting co-ordinate and direction.
    Calls check_space_free() to ensure boat placement is valid - if not, prompts user to place boat elsewhere.
    Saves placed boats in player_boats list."""
    
    for boat, data in boat_types.items():
        length, symbol = data
        coords = ""
        while not coords:            
            start_coord = coord_string_to_digit(enforce_player_coord(input(f"Enter start coordinate for {boat}: ").upper()))
            direction = ""
            while direction not in ["U","D","L","R"]:
                direction = input("Choose direction (U/D/L/R): ").upper()
            coords = check_space_free(sea_lst, start_coord, direction, length)
        player_boat_coords.append(coords)
        update_board_placement(sea_lst, coords, symbol)
        display_boards(radar_lst, sea_lst)

In [47]:
def check_hit(sea_lst, position):
    """Takes player guess position and returns True if that position in sea_lst was a boat."""
    print(sea_lst[position])
    if sea_lst[position] in sea_boats:
        return True       
    else:
        return False    

In [48]:
test_guess = 21

check_hit(sea_lst_p1, test_guess)

~~~


False

In [143]:
def check_ship_status(sea_lst, player_boat_coords, player_sunk_boats):
    """Checks which ships are destroyed and updates ship count."""
    for index, boat in enumerate(player_boat_coords):
        if all([sea_lst[boat[i]] == sea_hit for i in range(len(boat))]):
            player_sunk_boats.append(list(boat_types.keys())[index])
            
# Have running score on side?

In [121]:
list(enumerate(test_player_boats))

[(0, [1, 2, 3, 4, 5]),
 (1, [15, 16, 17, 18]),
 (2, [71, 81, 91]),
 (3, [45, 44, 43]),
 (4, [70, 60])]

In [126]:
test_sea = [sea_empty] * 101  
test_player_boats = []

place_boats(radar_lst_p1, test_sea, test_player_boats)

Radar zone

     1   2   3   4   5   6   7   8   9   10
A: |   |   |   |   |   |   |   |   |   |   |
B: |   |   |   |   |   |   |   |   |   |   |
C: |   |   |   |   |   |   |   |   |   |   |
D: |   |   |   |   |   |   |   |   |   |   |
E: |   |   |   |   |   |   |   |   |   |   |
F: |~~~|   |   |   |   |   |   |   |   |~x~|
G: |   |   |   |   |   |   |   |   |   |~x~|
H: |   |   |   |   |   |   |   |   |   |   |
I: |   |   |   |   |   |   |   |   |   |   |
J: |   |   |   |   |   |   |   |   |   |   |

Sea zone

     1   2   3   4   5   6   7   8   9   10
A: |~~~|~S~|~S~|~S~|~~~|~~~|~~~|~~~|~~~|~~~|
B: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
C: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~C~|~~~|~~~|
D: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~C~|~~~|~B~|
E: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~C~|~~~|~B~|
F: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~B~|
G: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~B~|
H: |~A~|~A~|~A~|~A~|~A~|~~~|~~~|~~~|~~~|~~~|
I: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
J: |~~~|~~~|~~~|~~~|~~~|~~~|~D~|~D

In [127]:
new_sea = test_sea

In [139]:
test_player_boats

[[71, 72, 73, 74, 75], [70, 60, 50, 40], [2, 3, 4], [28, 38, 48], [97, 98]]

In [82]:
test_guesses

['A5', 'G6', 'D4']

In [137]:
test_shunk = []
position = fire_shot(test_guesses)
flag = sea_empty
if check_hit(new_sea, position):
    flag = sea_hit

update_board_combat(radar_lst_p1, test_sea, position, flag)

Enter coordinates (in the format 'A1'):  h5


~A~


In [144]:
test_shunk = []
check_ship_status(new_sea, test_player_boats, test_shunk)
display_boards(radar_lst_p2, new_sea)
print(test_shunk)

Radar zone

     1   2   3   4   5   6   7   8   9   10
A: |   |   |   |   |   |   |   |   |   |   |
B: |   |   |   |   |   |   |   |   |   |   |
C: |   |   |   |   |   |   |   |   |   |   |
D: |   |   |   |   |   |   |   |   |   |   |
E: |   |   |   |   |   |   |   |   |   |   |
F: |   |   |   |   |   |   |   |   |   |   |
G: |   |   |   |   |   |   |   |   |   |   |
H: |   |   |   |   |   |   |   |   |   |   |
I: |   |   |   |   |   |   |   |   |   |   |
J: |   |   |   |   |   |   |   |   |   |   |

Sea zone

     1   2   3   4   5   6   7   8   9   10
A: |~~~|~S~|~S~|~S~|~~~|~~~|~~~|~~~|~~~|~~~|
B: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
C: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~C~|~~~|~~~|
D: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~C~|~~~|~B~|
E: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~C~|~~~|~B~|
F: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~B~|
G: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~B~|
H: |~x~|~x~|~x~|~x~|~x~|~~~|~~~|~~~|~~~|~~~|
I: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
J: |~~~|~~~|~~~|~~~|~~~|~~~|~x~|~x

In [114]:
for index, boat in enumerate(test_player_boats):
    print(index, boat)
    print([test_sea[boat[i]] == sea_hit for i in range(len(boat))])
    #all([sea_lst[boat[i]]] == sea_hit for i in range(len(boat)))

0 [1, 2, 3, 4, 5]
[False, False, False, False, False]
1 [15, 16, 17, 18]
[False, False, False, False]
2 [71, 81, 91]
[False, False, False]
3 [45, 44, 43]
[False, False, False]
4 [70, 60]
[True, True]


In [57]:
def winning_condition():
    """Returns true if either player has 0 ships remaining."""
    if len(player_2_remaining_boats) == 0 or len(player_1_remaining_boats) == 0:
        return True

    else:
        return False
        
    

In [51]:
def new_turn():
    """When turn ended (i.e. function is called), hides the previous player's board and asks next player if they are ready.
    Once second player is ready, sets turn to their go and displays their board."""
    pass

In [52]:
def final_result():
    """Displays both boards at the end of the game."""
    pass

In [54]:
replay_dropdown = widgets.Dropdown(
                    options=['Yes', 'No'],
                    value='Yes',
                    description='Replay? ',
                    disabled=False,
                )
def replay():
    """Asks players if they want to play again. If 'Yes', restarts game. If 'No', exits program."""
   if replay_dropdown.value == "Yes":
       
       
replay_dropdown
    
    


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 9)

In [55]:
print("\U0001F6E5")
print("\U000026F4")
print("\U0001F4A5")

🛥
⛴
💥
